In [1]:
import pandas_datareader as web
import pandas as pd
from yahoo_fin import stock_info as si
import datetime as dt
import os
#from MCForecastTools import MCSimulation

In [2]:
tickers = si.tickers_sp500(False)


In [3]:
start = dt.datetime.now() - dt.timedelta(days=365)
end = dt.datetime.now()

In [4]:
sp500_df = web.DataReader('^GSPC', 'yahoo', start, end)
sp500_df['Pct Change'] = sp500_df['Adj Close'].pct_change()
sp500_return = (((sp500_df['Pct Change'] + 1).cumprod())-1)[-1]

#sp500_df

In [5]:
return_list=[]

In [9]:
final_df = pd.DataFrame(columns=['Ticker', 'Latest_Price', 'Score', 'PE_Ratio', 'PEG_Ratio', 'SMA_150', 'SMA_200', '52_Week_Low', '52_Week_High'])

In [10]:
#make sure to have 'stock_data' folder in directory

for ticker in tickers:
    df = web.DataReader(ticker, 'yahoo', start, end)
    df.to_csv(f'stock_data/{ticker}.csv')
    
    df['Pct Change'] = df['Adj Close'].pct_change()
    stock_return = (((df['Pct Change'] + 1).cumprod())-1)[-1]
    
    returns_compared = round((stock_return / sp500_return), 2)
    return_list.append(returns_compared)
print(return_list)

FileNotFoundError: [Errno 2] No such file or directory: 'stock_data/A.csv'

In [11]:
"""
Original Code

best_performers = pd.DataFrame(list(zip(tickers, return_list)), columns=['Ticker', 'Returns Compared'])
best_performers['Score'] = best_performers['Returns Compared'].rank(pct=True) * 100
best_performers = best_performers[best_performers['Score'] >= best_performers['Score'].quantile(0.9)]
display(best_performers)
"""

best_performers = pd.DataFrame(list(zip(tickers, return_list)), columns=['Ticker', 'Returns Compared'])
best_performers['Score'] = best_performers['Returns Compared'].rank(pct=True) *100
best_performers = best_performers[best_performers['Returns Compared'] >= 0]
best_performers = best_performers[best_performers['Score'] >= best_performers['Score'].quantile(0.95)]
best_performers = best_performers.sort_values(by='Score', ascending=False)
display(best_performers)

,Ticker,Returns Compared,Score


In [12]:
#exmaple of locating pe_ratio and peg_ratio, values not available at certain times
pe_ratio = si.get_quote_table('OXY')
display(pe_ratio)
peg_ratio = float(si.get_stats_valuation('OXY').iloc[4][1])
display(peg_ratio)

{'1y Target Est': 76.74,
 '52 Week Range': '25.07 - 77.13',
 'Ask': '64.31 x 1200',
 'Avg. Volume': 26227331.0,
 'Beta (5Y Monthly)': 1.86,
 'Bid': '64.20 x 800',
 "Day's Range": '63.25 - 64.82',
 'EPS (TTM)': 10.17,
 'Earnings Date': 'Nov 02, 2022 - Nov 07, 2022',
 'Ex-Dividend Date': 'Sep 09, 2022',
 'Forward Dividend & Yield': '0.52 (0.81%)',
 'Market Cap': '59.876B',
 'Open': 64.1,
 'PE Ratio (TTM)': 6.32,
 'Previous Close': 64.68,
 'Quote Price': 64.27999877929688,
 'Volume': 20113308.0}

nan

In [13]:

for ticker in best_performers['Ticker']:
    try:
        df = pd.read_csv(f'stock_data/{ticker}.csv', index_col=0)
        moving_averages = [150, 200]
        for ma in moving_averages:
            df['SMA_' + str(ma)] = round(df['Adj Close'].rolling(window=ma).mean(), 2)
        latest_price =df['Adj Close'][-1]
        pe_ratio = float(si.get_quote_table(ticker)['PE Ratio (TTM)'])
        peg_ratio = float(si.get_stats_valuation(ticker).loc[4][1])
        moving_averages_150 = df['SMA_150'][-1]
        moving_averages_200 = df['SMA_200'][-1]
        low_52week = round(min(df['Low'][-(252):]), 2)
        high_52week = round(max(df['High'][-(252):]), 2)
        score = round(best_performers[best_performers['Ticker'] == ticker]['Score'].tolist()[0])

        condition_1 = latest_price > moving_averages_150 > moving_averages_200
        condition_2 = latest_price >= (1.3 * low_52week)
        condition_3 = latest_price >= (0.75 * high_52week)
        condition_4 = pe_ratio < 40
        condition_5 = peg_ratio < 2

        if condition_1 and condition_2 and condition_3:
            final_df = final_df.append({'Ticker': ticker,
                                        'Latest_Price': latest_price,
                                        'Score': score,
                                        'PE_Ratio': pe_ratio,
                                        'PEG_Ratio': peg_ratio,
                                        'SMA_150': moving_averages_150,
                                        'SMA_200': moving_averages_200,
                                        '52_Week_Low': low_52week,
                                        '52_Week_High': high_52week}, ignore_index=True)
    except Exception as e:
        print(f"{e} for {ticker}")

In [14]:
final_df.sort_values(by='Score', ascending=False)
#pd.set_option('display.max_columns', 10)
print(final_df)
final_df.to_csv('final.csv')

Empty DataFrame
Columns: [Ticker, Latest_Price, Score, PE_Ratio, PEG_Ratio, SMA_150, SMA_200, 52_Week_Low, 52_Week_High]
Index: []


In [144]:
#Tried monte carlo simulation, but keep running into error "IndexError: Too many levels: Index has only 1 level, not 2"
"""for ticker in final_df['Ticker']:
    #df = web.DataReader(ticker, 'yahoo', start, end)
    df = pd.read_csv(f'stock_data/{ticker}.csv')
    """index_col="Date",
        parse_dates=True,
        infer_datetime_format=True)"""
    #df = df.reset_index(inplace=False)
    df = df.set_index('Date')
    df = df.drop(['Adj Close'], axis=1)
    #df = df.reset_index(col_level=1)
    df['daily_return']= df['Close'].pct_change()
    df.columns.name = df.index.name
    df.index.name = None
    display(df)
    
    MC_threeyear = MCSimulation(
    portfolio_data = df,
    weights = [1],
    num_simulation = 500,
    num_trading_days = 756
    )
    
    #display(MC_threeyear.calc_cumulative_return())
   # display(MC_fiveyear.plot_simulation())"""

Date,High,Low,Open,Close,Volume,daily_return
2021-09-20,25.920000,24.959999,25.740000,25.480000,20470000.0,NaN
2021-09-21,26.080000,25.070000,25.820000,25.430000,12866200.0,-0.001962
2021-09-22,27.260000,26.219999,26.219999,26.750000,21793400.0,0.051907
2021-09-23,28.200001,26.580000,26.969999,27.959999,17428000.0,0.045234
2021-09-24,28.450001,27.549999,27.620001,28.100000,13473800.0,0.005007
...,...,...,...,...,...,...
2022-09-12,67.809998,65.680000,67.000000,66.800003,26349100.0,0.018138
2022-09-13,66.779999,64.519997,65.629997,65.339996,25446400.0,-0.021856
2022-09-14,67.790001,65.760002,66.220001,67.199997,28407000.0,0.028466
2022-09-15,66.139999,65.010002,65.800003,65.089996,24382100.0,-0.031399


IndexError: Too many levels: Index has only 1 level, not 2